In [1]:
import re
import numpy as np
import collections
from sklearn import metrics
from sklearn.cross_validation import train_test_split
import tensorflow as tf
import pandas as pd
from unidecode import unidecode
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import time

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
import json

with open('tokenized.json') as fopen:
    dataset = json.load(fopen)
texts = dataset['x']
labels = dataset['y']
del dataset

In [6]:
x, y = [], []
for i in tqdm(range(len(texts))):
    s = ' '.join(texts[i])
    if len(s) > 5:
        x.append(s)
        y.append(labels[i])

100%|██████████| 192029/192029 [00:00<00:00, 582639.90it/s]


In [7]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensorValue(indices, coo.col, coo.shape), tf.SparseTensorValue(indices, coo.data, coo.shape)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
bow_chars = CountVectorizer(ngram_range=(3, 5), analyzer='char_wb', max_features=300000).fit(x)
delattr(bow_chars, 'stop_words_')

In [11]:
vectors = bow_chars.transform(x)
feature_shape = vectors.shape[1]
feature_shape

300000

In [12]:
class Model:
    def __init__(self, output_size, vocab_size, learning_rate):
        self.X = tf.sparse_placeholder(tf.int32)
        self.W = tf.sparse_placeholder(tf.int32)
        self.Y = tf.placeholder(tf.float32, [None, output_size])
        embeddings = tf.Variable(tf.truncated_normal([vocab_size,128]))
        embed = tf.nn.embedding_lookup_sparse(embeddings, self.X, self.W, combiner='mean')
        self.logits = tf.layers.dense(embed, output_size)
        self.cost = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_prediction = tf.equal(tf.round(tf.nn.sigmoid(self.logits)), tf.round(self.Y))
        all_labels_true = tf.reduce_min(tf.cast(correct_prediction, tf.float32), 1)
        self.accuracy = tf.reduce_mean(all_labels_true)

In [13]:
sess = tf.InteractiveSession()
model = Model(6, feature_shape, 1e-4)
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [14]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'fast-text-char/model.ckpt')

'fast-text-char/model.ckpt'

In [15]:
train_X, test_X, train_Y, test_Y = train_test_split(
    vectors, y, test_size = 0.2
)

In [16]:
from tqdm import tqdm
import time

batch_size = 60
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, train_X.shape[0], batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = convert_sparse_matrix_to_sparse_tensor(train_X[i : min(i + batch_size, train_X.shape[0])])
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x[0],
                model.W: batch_x[1],
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    pbar = tqdm(range(0, test_X.shape[0], batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = convert_sparse_matrix_to_sparse_tensor(test_X[i : min(i + batch_size, test_X.shape[0])])
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x[0],
                model.W: batch_x[1],
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= train_X.shape[0] / batch_size
    train_acc /= train_X.shape[0] / batch_size
    test_loss /= test_X.shape[0] / batch_size
    test_acc /= test_X.shape[0] / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 3/2556 [00:00<01:52, 22.65it/s, accuracy=0.817, cost=0.204]

epoch: 0, pass acc: 0.000000, current acc: 0.899579
time taken: 115.20369625091553
epoch: 0, training loss: 0.286168, training acc: 0.839196, valid loss: 0.127799, valid acc: 0.899579



train minibatch loop:   0%|          | 3/2556 [00:00<01:52, 22.65it/s, accuracy=0.833, cost=0.165]

epoch: 1, pass acc: 0.899579, current acc: 0.900780
time taken: 115.12048959732056
epoch: 1, training loss: 0.116865, training acc: 0.897905, valid loss: 0.105463, valid acc: 0.900780



train minibatch loop:   0%|          | 3/2556 [00:00<01:52, 22.73it/s, accuracy=0.85, cost=0.137] 

...

time taken: 114.87326407432556
epoch: 38, training loss: 0.059900, training acc: 0.918582, valid loss: 0.071280, valid acc: 0.910386



train minibatch loop:   0%|          | 3/2556 [00:00<01:52, 22.63it/s, accuracy=0.867, cost=0.11]

time taken: 114.86839842796326
epoch: 39, training loss: 0.059557, training acc: 0.919038, valid loss: 0.071303, valid acc: 0.910491



test minibatch loop: 100%|██████████| 639/639 [00:03<00:00, 212.99it/s, accuracy=0.837, cost=0.153] 

time taken: 114.87353873252869
epoch: 40, training loss: 0.059221, training acc: 0.919567, valid loss: 0.071332, valid acc: 0.910386

break epoch:41



In [17]:
stack = []

pbar = tqdm(
    range(0, test_X.shape[0], batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = convert_sparse_matrix_to_sparse_tensor(test_X[i : min(i + batch_size, test_X.shape[0])])
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    stack.append(sess.run(
        tf.nn.sigmoid(model.logits), 
        feed_dict = {model.X: batch_x[0], model.W: batch_x[1], model.Y: batch_y}
    ))

validation minibatch loop: 100%|██████████| 639/639 [00:07<00:00, 84.25it/s]


In [18]:
print(metrics.classification_report(np.array(test_Y),np.around(np.concatenate(stack,axis=0)),
                                    target_names=["toxic", "severe_toxic", "obscene", 
                                            "threat", "insult", "identity_hate"]))

               precision    recall  f1-score   support

        toxic       0.83      0.56      0.67      3654
 severe_toxic       0.47      0.22      0.30       387
      obscene       0.84      0.54      0.65      1985
       threat       0.48      0.17      0.25       120
       insult       0.73      0.44      0.55      1862
identity_hate       0.44      0.18      0.26       326

  avg / total       0.77      0.49      0.60      8334



In [19]:
saver.save(sess, 'fast-text-char/model.ckpt')

'fast-text-char/model.ckpt'

In [20]:
import pickle
with open('vectorizer-sparse-toxicity.pkl','wb') as fopen:
    pickle.dump(bow_chars, fopen)